## Module 3 HW

---

### Question 1

What is count of records for the 2024 Yellow Taxi Data? 

Answer:

---

### Question 2

What is the estimated amount of data that will be read when this query is executed on the External Table and the Table?

Answer:

---

### Question 3

Why are the estimated number of Bytes different?

Answer:

---

### Question 4

How many records have a fare_amount of 0?

Answer:

---

### Question 5

What is the best strategy to make an optimized table in Big Query if your query will always filter based on tpep_dropoff_datetime and order the results by VendorID (Create a new table with this strategy)

Answer:

___

### Question 6

Write a query to retrieve the distinct VendorIDs between tpep_dropoff_datetime 2024-03-01 and 2024-03-15 (inclusive). Use the materialized table you created earlier in your from clause and note the estimated bytes. Now change the table in the from clause to the partitioned table you created for question 5 and note the estimated bytes processed. What are these values?

Answer:

---

### Question 7

Where is the data stored in the External Table you created?

Answer:

---

### Question 8

It is best practice in Big Query to always cluster your data:

Answer:

---

### Question 9

Write a `SELECT count(*)` query FROM the materialized table you created. How many bytes does it estimate will be read? Why?

Answer:

---